# Working with multiple phases

This notebook will explain how to load, access and fit multiple phases

#### Import Python packages

In [ ]:
# easyScience, technique-independent
import numpy as np
import easydiffraction as ed

# Structure visualization
import py3Dmol
# Charts visualization
from bokeh.io import output_notebook
from bokeh.io import show
from bokeh.plotting import figure

In [ ]:
output_notebook()
FIGURE_WIDTH = 990
FIGURE_HEIGHT = 300

## --- Sample ---

#### Show a CIF file content

In [ ]:
cif_fname = 'multiphase.cif'

with open(cif_fname, 'r') as f:
    content = f.read()
    
print(content)

 This cif file contains two phases of Si3N4: `alpha` and `beta`.

#### Create a job and load structure from a CIF file

In [ ]:
job = ed.Job()
job.add_sample_from_file(cif_fname)

phase_alpha = job.phases[0]
phase_beta = job.phases[1]

print(job.phases)
print(phase_alpha)
print(phase_beta)

#### Visualise the first phase

In [ ]:
structure = py3Dmol.view()
structure.addModel(phase_alpha.cif, 'cif')
structure.setStyle({'sphere':{'colorscheme':'Jmol','scale':.2},'stick':{'colorscheme':'Jmol','radius': 0.1}})
structure.addUnitCell()
structure.replicateUnitCell(2,2,1)
structure.zoomTo()

#### Visualise the second phase

In [ ]:
structure_2 = py3Dmol.view()
structure_2.addModel(phase_beta.cif, 'cif')
structure_2.setStyle({'sphere':{'colorscheme':'Jmol','scale':.2},'stick':{'colorscheme':'Jmol','radius': 0.1}})
structure_2.addUnitCell()
structure_2.replicateUnitCell(2,2,1)
structure_2.zoomTo()

## --- Experiment ---

#### Load the measured data

In [ ]:
meas_fname = '3T2@LLB.xye'
meas_x, meas_y, meas_e = np.loadtxt(meas_fname, unpack=True)

job.add_experiment_from_file(meas_fname)

#### Visualize the measured data

In [ ]:
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(meas_x, meas_y, legend_label='Experiment', color='orangered', line_width=2)
show(fig)

In [ ]:
meas_y

## --- Analysis ---

In [ ]:
print(f"Current calculator engine: {job.calculator}")

#### Generate the calculated data

**Note**: *Calculated data corresponds to the sum of all phases*

In [ ]:
calc_y_cryspy = job.calculate_profile()

#### Visualize both the measured and calculated data

In [ ]:
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(meas_x, meas_y, legend_label='Imeas', color='orange', line_width=2)
fig.line(meas_x, calc_y_cryspy, legend_label='Icalc (CrysPy)', color='blue', line_width=2)
show(fig)

#### We can also view separate phases contributions

In [ ]:
y_phase_1 = job.interface.get_calculated_y_for_phase(0)
y_phase_2 = job.interface.get_calculated_y_for_phase(1)

In [ ]:
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(meas_x, y_phase_1, legend_label='Si3N4 alpha', color='orange', line_width=2)
fig.line(meas_x, y_phase_2, legend_label='Si3N4 beta', color='blue', line_width=2)
show(fig)

#### Set scale manually, for each phase separately

In [ ]:
job.phases[0].scale = 92.
job.phases[1].scale = 28.6

#### Set wavelength manually

In [ ]:
job.parameters.wavelength = 1.2251

In [ ]:
calc_y_cryspy = job.calculate_profile()

calc_y_cryspy = job.interface.get_total_y_for_phases()[1]
calc_y_cryspy = job.interface.get_calculated_y_for_phase(1)


fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(meas_x, meas_y, legend_label='Imeas', color='orange', line_width=2)
fig.line(meas_x, calc_y_cryspy, legend_label='Icalc (CrysPy)', color='blue', line_width=2)
show(fig)

#### Set background points manually

In [ ]:
p1 = (job.experiment.x.data[0], 200)
p2 = (job.experiment.x.data[-1], 250)
points = [p1, p2]

job.set_background(points)

In [ ]:
calc_y_cryspy = job.calculate_profile()

fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(meas_x, meas_y, legend_label='Imeas', color='orange', line_width=2)
fig.line(meas_x, calc_y_cryspy, legend_label='Icalc (CrysPy)', color='blue', line_width=2)
show(fig)

#### Define parameters to optimize

In [ ]:
job.phases[0].scale.fixed = False
job.phases[1].scale.fixed = False
job.pattern.zero_shift.fixed = False
job.parameters.resolution_u.fixed = False
job.parameters.resolution_v.fixed = False
job.parameters.resolution_w.fixed = False
job.pattern.backgrounds[0][0].y.fixed = False
job.pattern.backgrounds[0][1].y.fixed = False

In [ ]:
print(job.phases[0].scale)
print(job.phases[1].scale)
print(job.pattern.zero_shift)
print(job.parameters.resolution_u)
print(job.parameters.resolution_v)
print(job.parameters.resolution_w)
print(job.pattern.backgrounds[0][0])
print(job.pattern.backgrounds[0][1])

#### Perform the fit

In [ ]:
print(f"Available minimizers: {job.analysis.available_minimizers}")
print(f"Current minimizer: {job.analysis.current_minimizer}")

In [ ]:
job.fit(method='least_squares', minimizer_kwargs={'diff_step': 1e-5})

result = job.fitting_results

In [ ]:
print("The fit has been successful: {}".format(result.success))
if result.success:    
    print("The gooodness of fit (chi2) is: {}".format(result.reduced_chi))
    print(job.pattern.scale)
    print(job.pattern.zero_shift)
    print(job.parameters.resolution_u)
    print(job.parameters.resolution_v)
    print(job.parameters.resolution_w)
    print(job.pattern.backgrounds[0][0])
    print(job.pattern.backgrounds[0][1])

In [ ]:
calc_y_cryspy = job.calculate_profile()

fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(meas_x, meas_y, legend_label='Imeas', color='orange', line_width=2)
fig.line(meas_x, calc_y_cryspy, legend_label='Icalc (CrysPy)', color='blue', line_width=2)
fig.line(meas_x, meas_y-calc_y_cryspy, legend_label='Imeas - Icalc (CrysPy)', color='green', line_width=2)
show(fig)